In [1]:
import os
from langgraph.graph import StateGraph, START, END
from langchain_groq import ChatGroq
from typing import TypedDict
from langgraph.checkpoint.memory import InMemorySaver
from dotenv import load_dotenv

In [2]:
# Load environment variables from .env file
load_dotenv()

# Get the API key from environment variables
groq_api_key = os.getenv("GROQ_API_KEY")

In [3]:
# Initialize the Groq model
model = ChatGroq(
    api_key=groq_api_key,
    model="llama-3.3-70b-versatile",
    temperature=0.7,          # Adjust for response randomness
)

In [4]:
class JokeState(TypedDict):

    topic: str
    joke: str
    explanation: str

In [6]:
def joke_generate(state: JokeState):

    prompt = f'Generate a joke on this topic {state["topic"]}'
    response = model.invoke(prompt).content

    return {'joke': response}

In [7]:
def joke_explain(state: JokeState):

    prompt = f'Write an explanation of this joke {state["joke"]}'
    response = model.invoke(prompt).content

    return {'explanation': response}

In [8]:
graph = StateGraph(JokeState)

graph.add_node('joke_generate', joke_generate)
graph.add_node('joke_explain', joke_explain)

graph.add_edge(START, 'joke_generate')
graph.add_edge('joke_generate', 'joke_explain')
graph.add_edge('joke_explain', END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [12]:
initial_state = {
    'topic': 'pizza'
}

config1 = {'configurable': {'thread_id': '1'}}

workflow.invoke(initial_state, config=config1)

{'topic': 'pizza',
 'joke': 'Why was the pizza in a bad mood?\n\nBecause it was feeling a little crusty!',
 'explanation': 'A classic play on words. This joke is an example of a pun, which is a type of wordplay that exploits multiple meanings of a word or phrase. In this case, the joke relies on the double meaning of the word "crusty."\n\nIn one sense, "crusty" can refer to the outer layer of a pizza, which is typically crispy and crunchy. However, "crusty" can also be used to describe someone or something that is irritable, grumpy, or in a bad mood. For example, you might say that someone is "feeling a little crusty" if they\'re being short-tempered or snappish.\n\nThe joke works by setting up the expectation that the pizza is in a bad mood, and then subverting that expectation by providing a clever and humorous reason for its bad mood. The punchline "it was feeling a little crusty" is a clever play on words, as it references both the physical properties of the pizza\'s crust and the 

In [13]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why was the pizza in a bad mood?\n\nBecause it was feeling a little crusty!', 'explanation': 'A classic play on words. This joke is an example of a pun, which is a type of wordplay that exploits multiple meanings of a word or phrase. In this case, the joke relies on the double meaning of the word "crusty."\n\nIn one sense, "crusty" can refer to the outer layer of a pizza, which is typically crispy and crunchy. However, "crusty" can also be used to describe someone or something that is irritable, grumpy, or in a bad mood. For example, you might say that someone is "feeling a little crusty" if they\'re being short-tempered or snappish.\n\nThe joke works by setting up the expectation that the pizza is in a bad mood, and then subverting that expectation by providing a clever and humorous reason for its bad mood. The punchline "it was feeling a little crusty" is a clever play on words, as it references both the physical properties of the pizz

In [14]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why was the pizza in a bad mood?\n\nBecause it was feeling a little crusty!', 'explanation': 'A classic play on words. This joke is an example of a pun, which is a type of wordplay that exploits multiple meanings of a word or phrase. In this case, the joke relies on the double meaning of the word "crusty."\n\nIn one sense, "crusty" can refer to the outer layer of a pizza, which is typically crispy and crunchy. However, "crusty" can also be used to describe someone or something that is irritable, grumpy, or in a bad mood. For example, you might say that someone is "feeling a little crusty" if they\'re being short-tempered or snappish.\n\nThe joke works by setting up the expectation that the pizza is in a bad mood, and then subverting that expectation by providing a clever and humorous reason for its bad mood. The punchline "it was feeling a little crusty" is a clever play on words, as it references both the physical properties of the piz

In [19]:
initial_state = {
    'topic': 'coffee'
}

config2 = {'configurable': {'thread_id': '2'}}

workflow.invoke(initial_state, config=config2)

{'topic': 'coffee',
 'joke': 'Why did the coffee file a police report?\n\nBecause it got mugged!',
 'explanation': 'A classic play on words. This joke is funny because it uses a clever pun to create a wordplay. Here\'s how it works:\n\nThe phrase "got mugged" has a double meaning. In one sense, "mugged" can mean to be robbed or attacked, which is why someone might file a police report. However, in this joke, "mugged" also refers to the fact that coffee is often served in a mug.\n\nSo, when the joke says "the coffee file a police report because it got mugged," it\'s making a humorous connection between the idea of being robbed (filing a police report) and the fact that coffee is typically contained in a mug. It\'s a lighthearted and clever play on words that creates a unexpected twist, making it amusing and entertaining.'}

In [20]:
workflow.get_state(config2)

StateSnapshot(values={'topic': 'coffee', 'joke': 'Why did the coffee file a police report?\n\nBecause it got mugged!', 'explanation': 'A classic play on words. This joke is funny because it uses a clever pun to create a wordplay. Here\'s how it works:\n\nThe phrase "got mugged" has a double meaning. In one sense, "mugged" can mean to be robbed or attacked, which is why someone might file a police report. However, in this joke, "mugged" also refers to the fact that coffee is often served in a mug.\n\nSo, when the joke says "the coffee file a police report because it got mugged," it\'s making a humorous connection between the idea of being robbed (filing a police report) and the fact that coffee is typically contained in a mug. It\'s a lighthearted and clever play on words that creates a unexpected twist, making it amusing and entertaining.'}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0716fd-a29d-68bc-8006-23c4edfb3dc0'}}, metadata={'sou

In [21]:
list(workflow.get_state_history(config2))

[StateSnapshot(values={'topic': 'coffee', 'joke': 'Why did the coffee file a police report?\n\nBecause it got mugged!', 'explanation': 'A classic play on words. This joke is funny because it uses a clever pun to create a wordplay. Here\'s how it works:\n\nThe phrase "got mugged" has a double meaning. In one sense, "mugged" can mean to be robbed or attacked, which is why someone might file a police report. However, in this joke, "mugged" also refers to the fact that coffee is often served in a mug.\n\nSo, when the joke says "the coffee file a police report because it got mugged," it\'s making a humorous connection between the idea of being robbed (filing a police report) and the fact that coffee is typically contained in a mug. It\'s a lighthearted and clever play on words that creates a unexpected twist, making it amusing and entertaining.'}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0716fd-a29d-68bc-8006-23c4edfb3dc0'}}, metadata={'so